In [1]:
# 來源: Python機器學習與深度學習特訓班,文淵閣工作室, 碁峯出版

In [2]:
import requests
from bs4 import BeautifulSoup as soup
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import jieba
import numpy as np
from collections import Counter

In [3]:
urls = []
url = 'https://udn.com/news/breaknews/1'
html = requests.get(url)
# 將網頁的HTML程式碼取回來後，接著引用BeautifulSoup類別(Class)，
# 傳入取回的HTML結構字串，並且指定HTML的解析型態來建立其物件
sp = soup(html.text, 'html.parser')
#data1 = sp.select('#breaknews_body dl dt h2 a')
# 單純只想要透過css屬性值來進行HTML節點的搜尋
data1 = sp.select("#breaknews div div div a")

In [4]:
data1

[<a aria-label="新冠肺炎全球確診逾6754萬例 超過154萬人病亡" class="story-list__image--holder" href="/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news"><picture><source data-srcset="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/wj/paper/2020/11/29/9161270.jpg&amp;s=Y&amp;x=0&amp;y=0&amp;sw=1279&amp;sh=853&amp;exp=3600&amp;w=300&amp;nt=1" type="image/webp"/><source data-srcset="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/wj/paper/2020/11/29/9161270.jpg&amp;s=Y&amp;x=0&amp;y=0&amp;sw=1279&amp;sh=853&amp;exp=3600&amp;w=300" type="image/jpeg"/><img alt="" class="lazyload" data-src="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/wj/paper/2020/11/29/9161270.jpg&amp;s=Y&amp;x=0&amp;y=0&amp;sw=1279&amp;sh=853&amp;exp=3600&amp;w=300"/></picture></a>,
 <a href="/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news" title="新冠肺炎全球確診逾6754萬例 超過154萬人病亡">新冠肺炎全球確診逾6754萬例 超過154萬人病亡</a>,
 <a href="/news/story/120944/5076871?from=udn-ch1_breaknews-1

In [5]:
for d in data1:
    urls.append('https://udn.com'+d.get('href'))

In [6]:
urls

['https://udn.com/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7320/5076877?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7320/5076877?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7320/5076877?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076861?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076861?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076861?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7266/5076832?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7266/5076832?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7266/5076832?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6656/5076872?from=udn-ch1_breaknews-1-0-news',


In [7]:
urls = urls[0::3]

In [8]:
urls

['https://udn.com/news/story/120944/5076871?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7320/5076877?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/120944/5076861?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7266/5076832?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6656/5076872?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/121837/5076867?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6812/5076844?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6885/5076866?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7002/5076865?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7321/5076864?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/7320/5076859?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6813/5076817?from=udn-ch1_breaknews-1-0-news',
 'https://udn.com/news/story/6812/5076826?from=udn-ch1_breaknews-1-0-news',
 'http

In [ ]:
text_news = ''
i = 1
for url in urls:
    html = requests.get(url)
    sp = soup(html.text, 'html.parser')
    #搜尋網頁中符合指定的HTML標籤及css屬性值的所有節點
    data1 = sp.find_all(class_ = "article-content__paragraph")
    print('處理第{}則新聞'.format(i))
    for d in data1:
        if d.text.find('延伸閱讀')!=-1:
            break
        if d.text != '':
            text_news += d.text
    i += 1

處理第1則新聞
處理第2則新聞
處理第3則新聞
處理第4則新聞
處理第5則新聞
處理第6則新聞
處理第7則新聞
處理第8則新聞
處理第9則新聞
處理第10則新聞


In [ ]:
text_news

In [ ]:
stops = []
with open('../week5/jieba_dictionary/stopwords.txt','r',encoding='utf-8') as file:
    for line in file.readlines():
        stops.append(line.strip())


# 文章自動摘要

In [ ]:
import summary as ausu

### 原理: 建立一個計算句子權重的函式，此函式包含關鍵詞數量、句子位置及句子相似度，計算文章中所有句子的權重，然後遞減排序，擷取使用者指定的句子數量，最後依據它們在原文中的先後順序再次進行排序輸出。

### tf-idf (term frequency - inverse document frequency):一種於資訊檢索與文字挖掘的常用加權技術。用以評估一個字詞對於一個檔案的重要程度。字詞的重要性會隨著它在檔案中出現的次數成正比增加

### summary模組包含下列方法
* split_sentence(text,punctuation_list='!?。！？')
* get_tfidf_matrix(sentence_set,stop_word)
* get_sentence_with_words_weight(tfidf_matrix)
* get_sentence_with_position_weight(sentence_set)
* get_similarity_weight(tfidf_matrix)
* ranking_base_on_wieght(sentence_with_words_weight,sentence_with_position_weight,sentence_score,feature_weight=[1,1,1])
* get_summarization(sentence_with_index,sort_sent_weight,topK_ratio=0.3)

In [ ]:
stops = []
with open('../week5/jieba_dictionary/stopwords.txt','r',encoding='utf-8') as file:
    for line in file.readlines():
        stops.append(line.strip())

import summary as ausu
text_news = ''
i = 1
for url in urls[0:5]:
    html = requests.get(url)
    sp = soup(html.text, 'html.parser')
    #搜尋網頁中符合指定的HTML標籤及css屬性值的所有節點
    data1 = sp.find_all(class_ = "article-content__paragraph")
    print('處理第{}則新聞'.format(i))
    for d in data1:
        if d.text.find('延伸閱讀')!=-1:
            break
        if d.text != '':
            text_news += d.text
    sentences, indexs = ausu.split_sentence(text_news)
    tfidf = ausu.get_tfidf_matrix(sentences, stops)
    word_weight = ausu.get_sentence_with_words_weight(tfidf)
    posi_weight = ausu.get_sentence_with_position_weight(sentences)
    scores = ausu.get_similarity_weight(tfidf)
    sort_weight = ausu.ranking_base_on_weight(word_weight,posi_weight,scores,feature_weight = [1,1,1])
    summar = ausu.get_summarization(indexs,sort_weight,topK_ratio=0.1) #取得摘要
    print('原文:\n',text_news)
    print('------------------------')
    print('摘要:\n',summar)
    print('=============================================')
    i += 1

In [ ]:
sentences, indexs

In [ ]:
# 下載 繁體中文詞庫  https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big
jieba.set_dictionary('./jieba_dictionary/dict.txt.big')
# 修改中文詞典，讓斷詞更聰明
with open('jieba_dictionary/stopwords.txt','r',encoding='utf-8') as file:
    stops = file.read().split('\n')
print('停用詞:'+'/'.join(stops))
terms  = []
for t in jieba.cut(text_news, cut_all = False):
    if t not in stops and t!='\n' and t!='\r\n':
        terms.append(t)
diction = Counter(terms)

In [ ]:
terms

In [ ]:
import imageio
back_color = imageio.imread('Taiwan.jpg')

unioncloud = WordCloud(
  background_color='white',        #   背景顏色
  max_words=200,                   #   最大分詞數量
  mask=back_color,                       #   背景圖片
  max_font_size=None,              #   顯示字體的最大值
  #stopwords=STOPWORDS.add(word),   #   使用內置的屏蔽詞，再添加一個
  font_path='./font/KAIU.ttc',             #   若為中文則需引入中文字型(.TTF)
  random_state=None,               #   隨機碼生成各分詞顏色
  prefer_horizontal=0.5)           #   調整分詞中水平和垂直的比例

In [ ]:
unioncloud.generate_from_frequencies(frequencies=diction) 

In [ ]:
import matplotlib.pyplot as plt
#plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.figure(figsize=(14,10))
plt.imshow(unioncloud)
plt.axis("off")
plt.show()